In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("du_lieu_chuan.csv")

In [3]:
df.head()

,Unnamed: 0,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,7590-VHVEG,0,0,1,0,1,0,2,1,...,0,0,0,0,0,1,0,29.85,29.85,0
1,1,5575-GNVDE,1,0,0,0,34,1,0,1,...,1,0,0,0,1,0,1,56.95,1889.50,0
2,2,3668-QPYBK,1,0,0,0,2,1,0,1,...,0,0,0,0,0,1,1,53.85,108.15,1
3,3,7795-CFOCW,1,0,0,0,45,0,2,1,...,1,1,0,0,1,0,2,42.30,1840.75,0
4,4,9237-HQITU,0,0,0,0,2,1,0,2,...,0,0,0,0,0,1,0,70.70,151.65,1


In [4]:
df.drop('customerID', axis=1, inplace=True)
df.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
X = df.drop("PaymentMethod", axis=1)
y = df["PaymentMethod"]

In [6]:
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [7]:
%%time
from sklearn.model_selection import GridSearchCV
# tạo tham số mô hình bằng grid
c_space = np.logspace(-5,10)
param_grid = {'C': c_space,  
              "penalty":['l1','l2'],
              'max_iter': [100,200,300,400,500],
              'tol':[0.0001,0.000001],      
             }

# mô hình logistic regression
logreg = LogisticRegression()

# khởi tạo logreg_cv chạy gridsearchCV
logreg_cv = GridSearchCV(logreg, param_grid=param_grid,cv=10,verbose=True, n_jobs=-1)

# chạy mô hình tốt nhất cho tập dữ liệu
best_model = logreg_cv.fit(X, y)

#in kết quả
print(best_model.best_score_)
print(best_model.best_params_)
print(best_model.best_estimator_)

Fitting 10 folds for each of 1000 candidates, totalling 10000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done 304 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-1)]: Done 804 tasks      | elapsed:   26.9s
[Parallel(n_jobs=-1)]: Done 1504 tasks      | elapsed:   54.3s
[Parallel(n_jobs=-1)]: Done 2182 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 2732 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 3382 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 4132 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done 4982 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 5932 tasks      | elapsed: 10.7min
[Parallel(n_jobs=-1)]: Done 6982 tasks      | elapsed: 13.5min
[Parallel(n_jobs=-1)]: Done 8132 tasks      | elapsed: 16.2min
[Parallel(n_jobs=-1)]: Done 9382 tasks      | elapsed: 19.3min
[Parallel(n_jobs=-1)]: Done 10000 out of 10000 | elapsed: 20.8min finished


0.4893344709897611
{'C': 0.04714866363457394, 'max_iter': 100, 'penalty': 'l2', 'tol': 1e-06}
LogisticRegression(C=0.04714866363457394, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=1e-06, verbose=0,
                   warm_start=False)
Wall time: 20min 47s


In [22]:
%%time
# tìm tham số tốt nhất với kernel = 'rbf' (default)
from sklearn.model_selection import GridSearchCV

def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs,
                  'gamma' : gammas,
                 'kernel' : ['rbf']}
    grid_search = GridSearchCV(SVC(), param_grid, cv=nfolds,scoring='accuracy')
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_

svc_param_selection(X,y,10)

Wall time: 14min 26s


{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}

In [23]:
svm = SVC()

In [24]:
%%time
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
# khởi tạo tham số mà muốn tìm
k_range = list(range(1, 100))

# khởi tạo tham số muốn tìm lên grid
param_grid = dict(n_neighbors=k_range)

# khởi tạo grid
grid = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy')
grid.fit(X, y)

Wall time: 34.8 s


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24, 25, 26, 27, 28, 29, 30, ...]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [25]:
# examine the best model
print(grid.best_score_)
print(grid.best_params_)
print(grid.best_estimator_)

0.9180887372013652
{'n_neighbors': 3}
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')


In [26]:
from sklearn.tree import DecisionTreeClassifier
DTC =DecisionTreeClassifier()
params = {'max_leaf_nodes': list(range(2, 100)),
          'min_samples_split': [2, 3, 4]}
grid_search_cv = GridSearchCV(DTC, params, cv=10)
grid_search_cv.fit(X, y)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=None,
                                              splitter='best'),
             iid='warn', n_jobs=None,
             param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15,

In [27]:
# examine the best model
print(grid_search_cv.best_score_)
print(grid_search_cv.best_params_)
print(grid_search_cv.best_estimator_)

0.996160409556314
{'max_leaf_nodes': 79, 'min_samples_split': 4}
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=79,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=4,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')
